<a href="https://colab.research.google.com/github/aman-welzin/rag/blob/main/GRAPH_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs
!pip install ollama gradio pymupdf langchain langchain-community langchain-core langchain-experimental langchain-huggingface transformers chromadb neo4j docling google-colab


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
!curl https://ollama.ai/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

!nvidia-smi

!nohup ollama serve &

# Check running models
!ollama ps

# Pull LLaMA 3.2 model (assuming the model is named 'llama3' in Ollama)
!ollama pull deepseek-coder:6.7b


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  61181      0 --:--:-- --:--:-- --:--:-- 61202
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Pack

In [4]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from typing import Tuple, List, Optional

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
import os
from langchain_community.graphs import Neo4jGraph
from langchain_community.graphs import Neo4jGraph # Make sure to import Neo4jGraph in the current cell


In [5]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass


from langchain_community.vectorstores import Neo4jVector

In [6]:
import ollama
import gradio as gr

from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma  # Handles storage and retrieval of vector embeddings using ChromaDB

# Embedding generation
from langchain_community.embeddings import OllamaEmbeddings  # Converts text into numerical vectors using Ollama's embedding model

import re  # Provides tools for working with regular expressions, useful for text cleaning and pattern matching

from typing import List

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain_community.llms import Ollama
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from transformers import pipeline
from google.colab import drive
from langchain_huggingface import HuggingFaceEmbeddings
from docling.document_converter import DocumentConverter
from langchain.schema import Document
from transformers import LlamaForCausalLM, LlamaTokenizer
from langchain.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document



In [7]:
!pip install -U langchain-neo4j
from langchain.graphs import Neo4jGraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 26.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [8]:
NEO4J_URI="neo4j+s://e3cb3331.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RzNdvZMnFteyovdi_A75W5Hj5S8gkd9Hnl26MxmtFdM"

In [9]:
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

graph = Neo4jGraph()

<ipython-input-9-8f5b0fd02ca0>:5: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [10]:
!pip install json-repair


In [11]:

def read_files():
  print("read all the files in directory")

  pdf_path = '/content/credit_card_details-ICICI.pdf'
  loader = PyMuPDFLoader(pdf_path)
  documents = loader.load()
  return documents

In [15]:
!ollama pull mistral


pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling ff82381e2bea:   0% ▕▏ 358 KB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   1% ▕▏  29 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   2% ▕▏  87 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   3% ▕▏ 140 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   4% ▕▏ 169 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   5% ▕▏ 202 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 239 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 241 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 242 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 243 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:

In [17]:
!pip install -U langchain-ollama

from langchain_ollama import OllamaLLM

# Use a lighter model like "mistral" or "gemma:2b" if you want faster generation
llm = OllamaLLM(model="mistral")


In [18]:
raw_document = read_files()
len(raw_document)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=5)
text_chunks  = text_splitter.split_documents(raw_document)
llm = OllamaLLM(model="mistral")
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(text_chunks)
graph_documents

read all the files in directory


[GraphDocument(nodes=[Node(id='Times Black ICICI Bank', type='Credit Card', properties={}), Node(id='Rs. 20000 + GST', type='Monetary Value', properties={}), Node(id='joining_fee', type='Fee', properties={}), Node(id='Not specified', type='Image URL', properties={}), Node(id='card_name', type='Card Name', properties={}), Node(id='Times Black ICICI Bank Credit Card', type='Credit Card', properties={}), Node(id='annual_fee', type='Fee', properties={}), Node(id='card_image', type='Image', properties={}), Node(id='bank_name', type='Bank Name', properties={})], relationships=[Relationship(source=Node(id='bank_name', type='Bank Name', properties={}), target=Node(id='Times Black ICICI Bank', type='Credit Card', properties={}), type='BELONGS_TO', properties={}), Relationship(source=Node(id='card_name', type='Card Name', properties={}), target=Node(id='Times Black ICICI Bank Credit Card', type='Credit Card', properties={}), type='IS', properties={}), Relationship(source=Node(id='joining_fee', t

In [38]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

# store this graph in a new table

In [70]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_index_similarity = Neo4jVector.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name="axis_credit_card_index",
    node_label="Document",
    text_node_property="page_content",  # property name in text_chunks
    embedding_node_property="_embedding_"
)

In [ ]:
# vector_index_similarity.update_index()
#    vector_index.update_index()

In [21]:
default_cypher = "MATCH (s)-[r:HAS_FEATURE|HAS_BENEFIT|HAS_FEE]->(t)RETURN s, r, t LIMIT 50"

In [22]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [23]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [24]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [52]:
vector_index = Neo4jVector.from_existing_graph(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],  # Changed from 'text' to 'page_content'
    embedding_node_property="embedding",
    graph=graph
)

In [87]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document as LangchainDocument

def hybrid_graph_rag_qa(query: str, vector_index, vector_index_similarity):

    # vector_index_similarity.update_index()
    # vector_index.update_index()

    retriever = vector_index.as_retriever(search_type="mmr",
                                         search_kwargs={"k": 2})
    # retriever = vector_index.as_retriever(search_kwargs={"k": 2})

    # retriever1 = vector_index_similarity.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    docs1 = retriever.get_relevant_documents(query)
    print(docs1)
    # docs2 = retriever1.get_relevant_documents(query)

    # combined_docs = docs1 + docs2
    # seen = set()
    # unique_docs = []
    # for doc in combined_docs:
    #     if doc.page_content not in seen:
    #         seen.add(doc.page_content)
    #         unique_docs.append(doc)

    combined_context = "\n\n".join([doc.page_content for doc in docs1])

    prompt = PromptTemplate.from_template("""
                        You are an intelligent assistant answering questions about credit cards
                        using the **official, pre-extracted, and verified information provided below**.

                        ✅ Use **only the context** to answer.
                        ❌ Do not say you lack internet access.
                        ❌ Do not refer the user to a financial advisor or a website.
                        ✅ Do not make assumptions — answer factually, even if partially.

                        📚 Context:
                        {context}

                        ❓ Question:
                        {question}

                        🧠 Answer:
                        """)

    llm = Ollama(model="deepseek-coder:6.7b")
    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run({
        "context": docs1,
        "question": query
    })

    print("🔎 Final Answer:\n", response)
    print("\n🧾 Sources:")
    # for i, doc in enumerate(unique_docs):
    #     print(f"\nSource {i + 1}:\n{doc.page_content}")

    return response

# hybrid_graph_rag_qa("What are the benefits of the Axis Magnus credit card?", vector_index, vector_index_similarity)

# hybrid_graph_rag_qa("What are the benefits of the MakeMyTrip ICICI Bank Signature Credit Card?", vector_index, vector_index_similarity)
# Card Card credit card?", vector_index, vector_index_similarity)

hybrid_graph_rag_qa("What are the top 3 cards for traveling?", vector_index, vector_index_similarity)


[Document(metadata={'page_content': 'card/images/adan\ni-platinum-card.\nwebp\nICICI\nMakeMyTrip ICICI Bank \nPlatinum Credit Card\nNA\nNA\nEarn 2 ICICI Reward points\n \n  for every Rs. 100 spent\n \n  except fuel\nJoining Fee: Rs.500+GST, Annual Fee (Second \nyear onwards)\n \n  â€“ Nil; Rs 500 My Cash plus MakeMyTrip \nholiday voucher worth Rs\n \n  3,000 as joining benefit; 1 complimentary airport \nlounge access per quarter and 1\n \n  complimentary railway\n \n  lounge access\nhttps://www.\nicicibank.\ncom/personal-\nbanking/cards/credi\nt-', 'moddate': '', 'keywords': '', 'subject': '', 'file_path': '/content/credit_card_details-ICICI.pdf', 'format': 'PDF 1.4', 'creationdate': '', 'creator': 'Google Sheets', 'creationDate': '', 'author': '', 'title': 'credit_card_details', 'trapped': '', 'source': '/content/credit_card_details-ICICI.pdf', 'modDate': '', 'page': 0, 'producer': '', 'total_pages': 1}, page_content='\ntext: card/images/adan\ni-platinum-card.\nwebp\nICICI\nMakeMyTrip

"Based on the context provided, here are three credit cards that could be beneficial for travelers:\n\n1. **ICICI Bank Platinum Credit Card**: This card offers a reward program of earning 2 ICICI Reward points for every Rs. 100 spent. However, it does not mention any joining fee or annual fee (Second year onwards). So, this could be an affordable option.\n\n2. **Axis Bank Platinum Credit Card**: This card is specifically designed for movie lovers and offers a significant cashback of up to 25%. It also includes complimentary airport lounge access per quarter and complimentary railway lounge access. However, it does not mention any joining fee or annual fee (Second year onwards).\n\n3. **Axis Bank Signature Credit Card with Lifestyle Benefits**: This card is more tailored towards those who enjoy travel and lifestyle shopping. It also offers a 25% cashback on movie related purchases. However, it does not mention any joining fee or annual fee (Second year onwards). As this information is o

In [89]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document as LangchainDocument

def hybrid_graph_rag_qa(query: str, vector_index, vector_index_similarity):

    # vector_index_similarity.update_index()
    # vector_index.update_index()

    # retriever = vector_index.as_retriever(search_type="mmr",
                                        #  search_kwargs={"k": 2})
    # retriever = vector_index.as_retriever(search_kwargs={"k": 2})

    retriever1 = vector_index_similarity.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    # docs1 = retriever.get_relevant_documents(query)
    docs2 = retriever1.get_relevant_documents(query)

    # combined_docs = docs1 + docs2
    # seen = set()
    # unique_docs = []
    # for doc in combined_docs:
    #     if doc.page_content not in seen:
    #         seen.add(doc.page_content)
    #         unique_docs.append(doc)

    combined_context = "\n\n".join([doc.page_content for doc in docs2])

    prompt = PromptTemplate.from_template("""
                        You are an intelligent assistant answering questions about credit cards
                        using the **official, pre-extracted, and verified information provided below**.

                        ✅ Use **only the context** to answer.
                        ❌ Do not say you lack internet access.
                        ❌ Do not refer the user to a financial advisor or a website.
                        ✅ Do not make assumptions — answer factually, even if partially.

                        📚 Context:
                        {context}

                        ❓ Question:
                        {question}

                        🧠 Answer:
                        """)

    llm = OllamaLLM(model="mistral")
    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run({
        "context": docs2,
        "question": query
    })

    print("🔎 Final Answer:\n", response)
    print("\n🧾 Sources:")
    # for i, doc in enumerate(combi):
    #     print(f"\nSource {i + 1}:\n{doc.page_content}")

    return response

# hybrid_graph_rag_qa("What are the benefits of the Axis Magnus credit card?", vector_index, vector_index_similarity)

# hybrid_graph_rag_qa("What are the benefits of the MakeMyTrip ICICI Bank Signature Credit Card?", vector_index, vector_index_similarity)
# Card Card credit card?", vector_index, vector_index_similarity)

hybrid_graph_rag_qa("What are the top 3 cards for traveling?", vector_index, vector_index_similarity)


🔎 Final Answer:
  Based on the provided data, it appears that there is only one credit card mentioned in the page content which is the Emeralde Credit Card by Nicici Bank. However, the page content does not provide specific details about the benefits of this card related to traveling. Therefore, I cannot definitively say which are the top 3 cards for traveling. It would be best to consult the official website or customer service of Nicici Bank for more information.

🧾 Sources:


' Based on the provided data, it appears that there is only one credit card mentioned in the page content which is the Emeralde Credit Card by Nicici Bank. However, the page content does not provide specific details about the benefits of this card related to traveling. Therefore, I cannot definitively say which are the top 3 cards for traveling. It would be best to consult the official website or customer service of Nicici Bank for more information.'

In [79]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document as LangchainDocument

def hybrid_graph_rag_qa(query: str, vector_index, vector_index_similarity):

    # vector_index_similarity.update_index()
    # vector_index.update_index()

    retriever = vector_index.as_retriever(search_type="mmr",
                                         search_kwargs={"k": 2})
    retriever = vector_index.as_retriever(search_kwargs={"k": 2})

    retriever1 = vector_index_similarity.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    docs1 = retriever.get_relevant_documents(query)
    docs2 = retriever1.get_relevant_documents(query)

    combined_docs = docs1 + docs2
    seen = set()
    unique_docs = []
    for doc in combined_docs:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            unique_docs.append(doc)

    combined_context = "\n\n".join([doc.page_content for doc in unique_docs])

    prompt = PromptTemplate.from_template("""
                        You are an intelligent assistant answering questions about credit cards
                        using the **official, pre-extracted, and verified information provided below**.

                        ✅ Use **only the context** to answer.
                        ❌ Do not say you lack internet access.
                        ❌ Do not refer the user to a financial advisor or a website.
                        ✅ Do not make assumptions — answer factually, even if partially.

                        📚 Context:
                        {context}

                        ❓ Question:
                        {question}

                        🧠 Answer:
                        """)

    llm = Ollama(model="deepseek-coder:6.7b")
    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run({
        "context": combined_context,
        "question": query
    })

    print("🔎 Final Answer:\n", response)
    print("\n🧾 Sources:")
    for i, doc in enumerate(unique_docs):
        print(f"\nSource {i + 1}:\n{doc.page_content}")

    return response

# hybrid_graph_rag_qa("What are the benefits of the Axis Magnus credit card?", vector_index, vector_index_similarity)

# hybrid_graph_rag_qa("What are the benefits of the MakeMyTrip ICICI Bank Signature Credit Card?", vector_index, vector_index_similarity)
# Card Card credit card?", vector_index, vector_index_similarity)

hybrid_graph_rag_qa("What are the top 3 cards for traveling?", vector_index, vector_index_similarity)


🔎 Final Answer:
 1. Axis Titanium Smart Traveler Credit Card: This card comes with a host of travel benefits like airport lounge access and railway lounge access per quarter, making it suitable for travelling. It also offers 8 points with every Rs.200 spent while travelling abroad, and a Fuel Surcharge waiver.

2. ICICI MakeMyTrip Platinum Credit Card: This card is known for its rich rewards programme which includes earning 2 ICICI Reward points for every Rs.100 spent, except fuel. It also offers a joining benefit of Rs.3,000 as well as a complimentary airport lounge access per quarter and railway lounge access.

3. ICICI Emirates Credit Card: This card is known for its rewards programme including earning Skywards miles on spends and unlimited complimentary Domestic & International lounge access. It also offers BOGO offer on movie & event tickets, Up to four times every month on tickets worth up to 750, and up to 4 rounds of complimentary golf basis the applicable spends on the previou

'1. Axis Titanium Smart Traveler Credit Card: This card comes with a host of travel benefits like airport lounge access and railway lounge access per quarter, making it suitable for travelling. It also offers 8 points with every Rs.200 spent while travelling abroad, and a Fuel Surcharge waiver.\n\n2. ICICI MakeMyTrip Platinum Credit Card: This card is known for its rich rewards programme which includes earning 2 ICICI Reward points for every Rs.100 spent, except fuel. It also offers a joining benefit of Rs.3,000 as well as a complimentary airport lounge access per quarter and railway lounge access.\n\n3. ICICI Emirates Credit Card: This card is known for its rewards programme including earning Skywards miles on spends and unlimited complimentary Domestic & International lounge access. It also offers BOGO offer on movie & event tickets, Up to four times every month on tickets worth up to 750, and up to 4 rounds of complimentary golf basis the applicable spends on the previous month.\n'

In [54]:
# import logging
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain.schema import Document as LangchainDocument

# # Setup logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# def validate_neo4j_index(graph):
#     """Validate that all Document nodes have embeddings."""
#     query = """
#     MATCH (d:Document)
#     WHERE d.embedding IS NULL
#     RETURN count(d) AS count
#     """
#     result = graph.query(query)
#     count = result[0]["count"]
#     if count > 0:
#         raise ValueError(f"Found {count} Document nodes without embeddings. Re-index the database.")
#     logger.info("Neo4j index validated successfully.")

# def deduplicate_documents(docs):
#     """Deduplicate documents based on content and metadata."""
#     seen = set()
#     unique_docs = []
#     for doc in docs:
#         content_hash = hash(doc.page_content + str(doc.metadata))
#         if content_hash not in seen:
#             seen.add(content_hash)
#             unique_docs.append(doc)
#     return unique_docs

# def hybrid_graph_rag_qa(query: str, vector_index, vector_index_similarity, graph):
#     """
#     Performs hybrid RAG-based question answering using vector and graph-based retrieval.

#     Args:
#         query: The user's question.
#         vector_index: Neo4jVector index for hybrid search.
#         vector_index_similarity: Neo4jVector index for similarity search.
#         graph: Neo4jGraph instance for database operations.

#     Returns:
#         The generated answer based on retrieved documents.
#     """
#     # Validate Neo4j index
#     try:
#         validate_neo4j_index(graph)
#     except Exception as e:
#         logger.error(f"Index validation failed: {str(e)}")
#         return "Failed to validate the Neo4j index."

#     # Retrieve documents
#     try:
#         retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})
#         retriever1 = vector_index_similarity.as_retriever(search_type="similarity", search_kwargs={"k": 5})
#         docs1 = retriever.get_relevant_documents(query)
#         docs2 = retriever1.get_relevant_documents(query)
#         logger.info(f"Retrieved {len(docs1)} documents from vector_index.")
#         logger.info(f"Retrieved {len(docs2)} documents from vector_index_similarity.")
#     except Exception as e:
#         logger.error(f"Error retrieving documents: {str(e)}")
#         return "Failed to retrieve relevant documents due to an indexing error."

#     # Deduplicate documents
#     combined_docs = docs1 + docs2
#     unique_docs = deduplicate_documents(combined_docs)
#     logger.info(f"Total unique documents: {len(unique_docs)}")

#     if not unique_docs:
#         logger.warning("No relevant documents retrieved.")
#         return "No relevant information found in the provided context."

#     combined_context = "\n\n".join([doc.page_content for doc in unique_docs])

#     # Prompt
#     prompt = PromptTemplate.from_template("""
#     You are an intelligent assistant answering questions about financial services
#     using the **official, pre-extracted, and verified information provided below**.

#     ✅ Use **only the context** to answer.
#     ❌ Do not say you lack internet access.
#     ❌ Do not refer the user to a financial advisor or a website.
#     ✅ If the context does not contain enough information to answer fully, state what is available and indicate that further details are not provided.

#     📚 Context:
#     {context}

#     ❓ Question:
#     {question}

#     🧠 Answer:
#     """)

#     # Initialize LLM
#     try:
#         llm = Ollama(model="deepseek-coder:6.7b")
#         chain = LLMChain(llm=llm, prompt=prompt)
#     except Exception as e:
#         logger.error(f"Failed to initialize Ollama model: {str(e)}")
#         return "Failed to initialize the language model."

#     # Generate response
#     try:
#         response = chain.run({"context": combined_context, "question": query})
#     except Exception as e:
#         logger.error(f"Error generating response: {str(e)}")
#         return "Failed to generate a response due to an LLM error."

#     logger.info("🔎 Final Answer:\n%s", response)
#     logger.info("\n🧾 Sources:")
#     for i, doc in enumerate(unique_docs):
#         logger.info(f"\nSource {i + 1}:\n{doc.page_content}")

#     return response

# # Example usage
# response = hybrid_graph_rag_qa(
#     "What are the benefits of the Axis Magnus credit card?",
#     vector_index,
#     vector_index_similarity,
#     graph  # Pass the Neo4jGraph instance
# )